In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline

In [53]:
args = {'lr':0.01,
        'epochs':1000,
        'noise':0.01,
        'train size':0.7,
        'spec scale':10**12,
        'stages':11,
        'hidden size':32
        }

In [54]:
df = pd.read_excel('wide_range.xlsx')

In [55]:
X = df.iloc[:,11:]
y = df.iloc[:,:11]

In [56]:
# scale by 10^12
X = X.apply(lambda x: x*args['spec scale'])

# add noise
noise = np.random.normal(loc=0, scale=args['noise'], size=X.shape)
X = X + noise

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=args['train size'], random_state=101)

In [58]:
class AttentionCNN(nn.Module):
    def __init__(self, input_size, output_size, num_layers):
        super(AttentionCNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(16, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

        self.shared_layers = nn.Sequential(
            nn.Linear(64 * self.calculate_conv_output_size(input_size), 128),
            nn.ReLU()
        )

        self.attention_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 1),
                nn.Softmax(dim=1)
            ) for _ in range(num_layers)
        ])

        self.prediction_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, output_size)
            ) for _ in range(num_layers)
        ])

    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv_layers(x)
        x = x.view(batch_size, -1)
        x = self.shared_layers(x)

        layer_outputs = []
        for layer in range(self.num_layers):
            attention_weights = self.attention_layers[layer](x)
            attention_input = torch.mul(x, attention_weights)
            layer_output = self.prediction_layers[layer](attention_input)
            layer_outputs.append(layer_output)

        return layer_outputs

    def calculate_conv_output_size(self, input_size):
        with torch.no_grad():
            x = torch.zeros(1, 1, input_size)
            x = self.conv_layers(x)
            return x.view(1, -1).shape[1]

In [59]:
input_size = 86
output_size = 1
num_layers = 11

In [60]:
model = AttentionCNN(input_size, output_size, num_layers)

In [61]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

In [62]:
X_train_array = X_train.values.reshape(-1, 1, input_size)
X_test_array = X_test.values.reshape(-1, 1, input_size)
y_train_array = y_train.values
y_test_array = y_test.values

# Convert to tensors
X_train_tensor = torch.Tensor(X_train_array)
X_test_tensor = torch.Tensor(X_test_array)
y_train_tensor = torch.Tensor(y_train_array)
y_test_tensor = torch.Tensor(y_test_array)

In [63]:
num_epochs = args['epochs']
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    layer_outputs = model(X_train_tensor)

    total_loss = 0
    for layer_output, y_train_layer in zip(layer_outputs, y_train_tensor.T):
        loss = criterion(layer_output.squeeze(), y_train_layer)
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if (epoch+1) % int(args['epochs']/10) == 0:
            print(f'epoch: {epoch+1}, total loss = {total_loss}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1540x576 and 36864x128)

In [ ]:
def plot_comparison(pred, actual):
    plt.title("Temperature Across Layers")
    plt.xlabel("Layer")
    plt.ylabel("Temperature")
    plt.plot(pred, 'r+', label='predicted')
    plt.plot(actual, 'g', label='actual')
    plt.show()